In [1]:
import pandas as pd
import os
from get_file_from_email import *
from db_funcs import *
from normalize_funcs import *
import config
import numpy as np
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)
db_name = config.db_name

# Файл для создания справочника weborama_accounts - собрал руками и сохранил в эксель
# это нужно только для 1 загрузки, чтобы потом сверяться с таблице из БД и проверять изменения
file_name = 'weborama_accounts.xlsx'
file_path = config.email_file_path

In [2]:
# создаем справочник источников full_source_types

# при первом подключении заюираем справочник Источников из MySQL
# необходимо его пересоздать в MSSQL
# при повторных подключениях будем работать именно с MSSQL
source_types = config.source_types #'source_types'
df_sources = get_mysql_full_dict_table(db_name, source_types)

df_sources = df_sources.drop(['created_at', 'updated_at'], axis=1)


# создаем общий список названий полей и типов данных 
# этот список передаем в БД MSSQL для создания новой таблицы
sources_vars_list = config.sources_vars_list

# создаем пустую таблицу Справочников в БД
full_source_types = config.full_source_types #'full_source_types'
createDBTable(db_name, full_source_types, sources_vars_list)

source_int_lst = config.source_int_lst
df_sources = normalize_columns_types(df_sources, source_int_lst)

# записываем справочник Источников в БД
# table_name = 'full_source_types'
downloadTableToDB(db_name, full_source_types, df_sources)

Все ок. Подключились!
Данные загружены
Все ок. Подключились!
Пустая таблица full_source_types успешно создана в БД tenant_perekrestok
Все ок. Подключились!
Скрипт запущен 2025-03-03 11:52:46.763027
Данные добавлены в БД: tenant_perekrestok, таблица: full_source_types
Скрипт отработал 2025-03-03 11:52:46.776026
Время выполнения задачи: 0:00:00.012999
Загрузка завершена. Данные успешно добавлены в БД: tenant_perekrestok
##################################################



In [ ]:
# Создаем справочник аккаунтов full_accounts_dict

accounts = config.accounts #'accounts'
df_accounts = get_mysql_full_dict_table(db_name, accounts)

source_types = config.source_types #'source_types'
df_sources = get_mysql_full_dict_table(db_name, source_types)

df_sources = df_sources.drop(['created_at', 'updated_at'], axis=1)
df_sources = df_sources.rename(columns={'id': 'source_type_id'})
df_accounts = df_accounts[['id', 'source_type_id', 'account_name', 'account_id', 'acc_id_flag']]
df_accounts['weborama_account_name'] = 'x5_perekrestok'
# # добавляем к справочнику Аккаунтов названия Источников
df_accounts = df_accounts.merge(df_sources[['source_type_id', 'utm_source_metrika']], how='left', left_on='source_type_id', right_on='source_type_id')
df_accounts = df_accounts.rename(columns={'utm_source_metrika': 'source'})

# забираем название для справочника аккаунтов MSSQL
full_accounts_dict = config.full_accounts_dict #'full_accounts_dict'


# создаем общий список названий полей и типов данных 
# этот список передаем в БД MSSQL для создания новой таблицы
full_accounts_vars_list = config.full_accounts_vars_list
# создаем пустую таблицу Аккаунтов в БД
createDBTable(db_name, full_accounts_dict, full_accounts_vars_list)

full_accounts_int_lst  = config.full_accounts_int_lst
df_accounts = normalize_columns_types(df_accounts, full_accounts_int_lst)

# записываем справочник Аккаунтов в БД
# table_name = 'full_accounts_dict'
downloadTableToDB(db_name, full_accounts_dict, df_accounts)

In [ ]:
# создаем таблицу Аккаунтов только Веборама weborama_accounts
weborama_accounts = config.weborama_accounts #'weborama_accounts'

# создаем общий список названий полей и типов данных 
# этот список передаем в БД MSSQL для создания новой таблицы
weborama_accounts_vars_lst = config.weborama_accounts_vars_lst

createDBTable(db_name, weborama_accounts, weborama_accounts_vars_lst)

df_weborama_accounts = pd.read_excel(os.path.join(file_path, file_name))
df_weborama_accounts = df_weborama_accounts[['source_type_id', 'account_name', 'id', 'source']]
df_weborama_accounts = df_weborama_accounts.rename(columns={'id': 'account_id'})

weborama_accounts_int_lst = config.weborama_accounts_int_lst
df_weborama_accounts = normalize_columns_types(df_weborama_accounts, weborama_accounts_int_lst)
df_weborama_accounts = df_weborama_accounts.sort_values('source_type_id')

# записываем справочник Аккаунтов Weborama в БД
# table_name = 'weborama_accounts'
downloadTableToDB(db_name, weborama_accounts, df_weborama_accounts)

In [ ]:
# Добавим аккаунты Веборамы к таблице всех Аккаунтов / перезаписываем full_accounts_dict

# Это тоже первая заливка, в дальнейшем будем добавлять только новые аккаунты
weborama_accounts = config.weborama_accounts #'weborama_accounts'
df_weborama_accounts = get_mssql_table(db_name, weborama_accounts)

full_accounts_dict = config.full_accounts_dict #'full_accounts_dict'
df_accounts = get_mssql_table(db_name, full_accounts_dict)


max_account_id = df_accounts['id'].max()+1 # забираем максимальный ИД из справояника MSSQL
# создаем нумерацию новых аккаунтов, начиная с последней записи из общего справочника аккаунтов
ids_list = [i for i in range(max_account_id, len(df_weborama_accounts)+max_account_id)]
# добавляем новое поле к таблице аккаунтов из Веборамы
df_weborama_accounts['id'] = pd.Series(ids_list)
# добавляем ведущий ноль к ИД аккаунта
df_weborama_accounts['acc_id_flag'] = df_weborama_accounts['id'].apply(lambda x: '0' + str(x) if len(str(x))<2 else str(x))
# добавляем поле с названием аккаунта, чтобы сошлись таблицы 
df_weborama_accounts['weborama_account_name'] = df_weborama_accounts['account_name']
# создаем итоговый датаФрейм для записи с Общим справочником аккаунтов
df_union_accounts = pd.concat([df_accounts, df_weborama_accounts])

# Общий Справочник Аккаунтов
# создаем общий список названий полей и типов данных 
# этот список передаем в БД MSSQL для создания новой таблицы

full_accounts_vars_list = config.full_accounts_vars_list

# записываем новую таблицу в БД
full_accounts_dict = config.full_accounts_dict #'full_accounts_dict'

createDBTable(db_name, full_accounts_dict, full_accounts_vars_list, flag='drop')
downloadTableToDB(db_name, full_accounts_dict, df_union_accounts)

In [3]:
# создаем cправочник Кампаний в БД weborama_camp_dict 
weborama_camp_dict = config.weborama_camp_dict #'weborama_camp_dict'

# Справочник Кампаний
# этот список передаем в БД MSSQL для создания новой таблицы
weborama_camp_dict_vars_lst  = config.weborama_camp_dict_vars_lst

# создаем пустую таблицу
createDBTable(db_name, weborama_camp_dict, weborama_camp_dict_vars_lst)

Все ок. Подключились!
Пустая таблица weborama_camp_dict успешно создана в БД tenant_perekrestok


['weborama_camp_name nvarchar(200)',
 'flight nvarchar(100)',
 'type nvarchar(100)',
 'category nvarchar(100)',
 'product nvarchar(100)',
 'source_type_id smallint',
 'source nvarchar(100)',
 'rotation_type nvarchar(10)',
 'main_acc_id smallint',
 'weborama_key_camp nvarchar(200)',
 'inner_campaign_id smallint',
 'date_start nvarchar(10)',
 'date_finish nvarchar(10)',
 'flight_name nvarchar(10)']

In [ ]:
 # Медиаплан в разбивке по дням weborama_plan_table
weborama_plan_table = config.weborama_plan_table #'weborama_plan_table'

# создаем общий список названий полей и типов данных 
# этот список передаем в БД MSSQL для создания новой таблицы
weborama_plan_table_vars_lst = config.weborama_plan_table_vars_lst

# создаем пустую таблицу Справочников в БД
createDBTable(db_name, weborama_plan_table, weborama_plan_table_vars_lst)

In [ ]:
# Ежедневный отчет Веборама weborama_report_table
weborama_report_table = config.weborama_report_table #'weborama_report_table'

# создаем общий список названий полей и типов данных 
# этот список передаем в БД MSSQL для создания новой таблицы
weborama_report_table_vars_lst = config.weborama_report_table_vars_lst

# пересоздаем пустую таблицу Справочников в БД
createDBTable(db_name, weborama_report_table, weborama_report_table_vars_lst)
# dropTable(db_name, table_name)